In [1]:
import pandas as pd
import numpy as np
import os

from itertools import count
import random
import matplotlib
from matplotlib import animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib qt

## Episodios

In [2]:
# leemos las ubicaciones de los episodios
pahts = os.listdir("../data/processed/episodios/")

# inicializamos un dataframe con la estructura definida
episodios = pd.DataFrame(columns=['name_podcast', 'publiser', 'id_publiser', 'total_episodios',
       'description', 'duration', 'id_episodio', 'language', 'name_episodio',
       'release_date', 'type'])

# concatenamos los episodios
for path in pahts:
    df_aux = pd.read_csv("../data/processed/episodios/"+path)
    episodios = pd.concat([episodios, df_aux])
    

# corregimos un error en la fecha
episodios = episodios[episodios["release_date"]!="0201-04-28"] 

# transformamos el tipo fecha y ordenamos los capitulos
episodios["release_date"] = pd.to_datetime(episodios["release_date"])
episodios.sort_values("release_date", inplace=True)

# eliminamos duplicados, reiniciamos los indixes y filtramos los capitulos que no esten en espanol.
episodios = episodios.drop_duplicates()
episodios = episodios.reset_index().drop(columns="index")
episodios = episodios[episodios.language.str.startswith("es", na = False)]

podcast_diarios = episodios[episodios.release_date<pd.to_datetime("2021-07-29")].groupby("release_date").publiser.count()
podcast_diarios.name="Podcast diarios"

podcast_mm = podcast_diarios.rolling(window=7).mean()
podcast_mm.name = "Media movil"

# filtramos los datos de 2015-actualidad
podcast_diarios= podcast_diarios[1784:]
podcast_mm = podcast_mm[1784:]

In [49]:
plt.style.use("seaborn-dark")
for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = '#000000'  # color de los ejes, y fondo.

for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '#00ff41'  # color de las etiquetas del marco

# tamano de la figura
fig, ax1 = plt.subplots(figsize=(10,6))

# parametros auxiliares
n_lines = 10
diff_linewidth = 1.05
alpha_value = 0.03
def animate(i):
    # filtramos los datos para cada iteracion
    data = podcast_diarios[:int(i+1)]
    data2 = podcast_mm[:int(i+1)]
    xp, y1, y2 = [], [], []

    lines = data
    
    # creamos las listas.
    for line in lines:
        xp = data.index
        y1 = data.to_list()
        y2 = data2.to_list()
    
    # limpiamos los graficos ax(1):
    ax1.clear()
    # episodios diarios
    plt.plot(xp, y1, linewidth=0.3, alpha=0.7, c="red", label="Número de episodios")
    # media movil de los episodios  
    plt.plot(xp, y2, linewidth=1, color="#00ff41", label="Media movil")
    plt.plot(xp[-1], y2[-1], marker="d", color="#00ff41", markersize=4)

    # graficamos nuevamente la media movil con margen.
    for n in range(1, n_lines+1):
        plt.plot(xp, y2,
            linewidth=2+(diff_linewidth*n),
            alpha=alpha_value)
        
    plt.legend(loc="upper left")
    plt.text(pd.to_datetime("2016-10-01"), 300,'twitter: @EnriqeSc', ha='right', alpha=0.3)
    plt.text(pd.to_datetime("2017-01-01"), 200,'Source: API de Spotify', ha='right', alpha=0.3)
    plt.title("Historia de los episodios en español.")
    
# creamos la animacion
anim = FuncAnimation(fig, animate, frames = len(podcast_diarios), interval=0.001)

# configuracion para guardar la animacion 'ffmpeg'
Writer = animation.writers['ffmpeg']
writer = Writer(fps=100, metadata=dict(artist='Enrique Santibáñez Cortés'))

# guargamos la animacion
#anim.save('../figure/episodios_complete.mp4', writer=writer)

## Podcasts

In [3]:
# seleccionamos la fecha del primer y el ultimo episodio de cada podcasts
fechas_de_nacimiento = episodios.groupby("id_publiser").head(1)[["id_publiser", "total_episodios", "language",
                                                               "release_date"]]
fechas_de_nacimiento.rename(columns={"release_date":"bird_date"}, inplace=True)

fechas_de_sepelio = episodios.groupby("id_publiser").tail(1)[["id_publiser", "total_episodios", "language",
                                                               "release_date"]]


# concatenamos la fechas anteriores.
fecha_crecimiento = pd.merge(fechas_de_nacimiento, fechas_de_sepelio, on=["id_publiser", "total_episodios",
                                                                          "language"])

# sumamos 7 semanas a la fecha del ultimo episodio y recortamos al 2021-07-29
fecha_crecimiento["release_date"] = fecha_crecimiento["release_date"]+pd.Timedelta(weeks=7)
fecha_crecimiento["release_date"] = fecha_crecimiento["release_date"].apply(lambda x: pd.to_datetime("2021-07-29") if x>pd.to_datetime("2021-07-28") else x)

# inicializamos el rango de las fechas
date_creados = pd.date_range(start=pd.to_datetime("2007-05-12"), end=pd.to_datetime("2021-07-28"), freq="D")

# inicializamos las listas para cada categoria
activos = [] # si la fecha del ultimo episodio es mas reciente
nacidos = [] # la fecha del primer episodio
cerrados = [] # si la fecha del ultimo episodio ya paso

for i in date_creados: # contamos los podcasts de cada categoria
    activos.append(fecha_crecimiento[(fecha_crecimiento.bird_date<=i) & 
                  (i<=fecha_crecimiento.release_date)].shape[0])
    nacidos.append(fecha_crecimiento[fecha_crecimiento.bird_date==i].shape[0])
    cerrados.append(fecha_crecimiento[fecha_crecimiento.release_date<i].shape[0])
    
# creamos el dataframe.
informacion_podcats = pd.DataFrame({"date":date_creados, "activos":activos, "nacidos":nacidos, "cerrados":cerrados})
informacion_podcats["nacidos_acumulados"] = informacion_podcats.nacidos.cumsum() # creamos los podcasts acumulados
informacion_podcats = informacion_podcats[informacion_podcats.date>=pd.to_datetime("2015-01-01")] # 2015 en adelante

In [47]:
plt.style.use("seaborn-dark")
for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
    plt.rcParams[param] = '#000000'  # color de los ejes, y fondo.

for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '#00ff41'  # color de las etiquetas del marco

# tamano de la figura
fig, ax1 = plt.subplots(figsize=(10,6))

# parametros auxiliares
n_lines = 10
diff_linewidth = 1.05
alpha_value = 0.03
def animate(i):
    # filtramos los datos para cada iteracion
    data = informacion_podcats.iloc[:i+1]
    
    xp, y1, y2, y3 = [], [], [], []
    lines = data.index.to_list()
    # creamos las listas.
    for line in lines:
        xp = data.index
        y1 = data["activos"].to_list()
        y2 = data["cerrados"].to_list()
        y3 = data["nacidos_acumulados"].to_list()
    
    # limpiamos los graficos ax(1):
    ax1.clear()
    
    # podcasts activos 
    plt.plot(xp, y1, linewidth=1, c="#FE53BB", label="Podcasts activos")
    plt.plot(xp[-1], y1[-1], marker="d", c="#FE53BB", markersize=4)
    
    # podcasts abandonados 
    plt.plot(xp, y2, linewidth=1, c="#08F7FE", label="Podcasts abandonados")
    plt.plot(xp[-1], y2[-1], marker="d", c="#08F7FE", markersize=4)
    
    # podcasts acumulados 
    plt.plot(xp, y3, linewidth=0.3, c="#00ff41", label="Podcasts acumulados")
    plt.plot(xp[-1], y3[-1], marker="d", c="#00ff41", markersize=4)

    # graficamos nuevamente los tres categorias para resaltar la original
    for n in range(1, n_lines+1):
        plt.plot(xp, y3, linewidth=2+(diff_linewidth*n), alpha=alpha_value, c="#00ff41")
        plt.plot(xp, y1, linewidth=2+(diff_linewidth*n), alpha=alpha_value, c="#FE53BB")
        plt.plot(xp, y2, linewidth=2+(diff_linewidth*n), alpha=alpha_value, c="#08F7FE")

        
    plt.legend(loc="upper left")
    plt.text(pd.to_datetime("2016-10-01"), 3000,'twitter: @EnriqeSc', ha='right', alpha=0.3)
    plt.text(pd.to_datetime("2017-01-01"), 2000,'Source: API de Spotify', ha='right', alpha=0.3)
    plt.title("Historia de los podcasts en espanol.")
    
# creamos la animacion
anim = FuncAnimation(fig, animate, frames = informacion_podcats.shape[0], interval=0.001)

# configuracion para guardar la animacion 'ffmpeg'
Writer = animation.writers['ffmpeg']
writer = Writer(fps=100, metadata=dict(artist='Enrique Santibáñez Cortés'))

# guargamos la animacion
#anim.save('../figure/podcasts_diarios.mp4', writer=writer)